# Testing out tensorflow
Source: https://www.tensorflow.org/versions/r0.10/tutorials/mnist/pros/index.html

## Simple logistic regression

In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import tensorflow as tf

# Set up an interactive session.  When not using iPython, you should use a regular (non-interactive) session.

sess = tf.InteractiveSession()

In [3]:
# Create placeholders to hold input and target

x = tf.placeholder(tf.float32, shape=[None, 784]) # Input: pixels of image (28*28 = 784)
y_ = tf.placeholder(tf.float32, shape=[None, 10]) # Target: one hot encoder indicating which digit

In [4]:
# Create variables to hold model weights and bias.  They are initialized to 0s.

W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [9]:
# Tensorflow doesn't actually run anything until you tell it to...
sess.run(tf.initialize_all_variables())

In [10]:
# Define output function as softmax (sigmoid)
y = tf.nn.softmax(tf.matmul(x,W) + b)

In [11]:
# Define error function
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [12]:
# Create object to represent training step.  Will train when executed.

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [13]:
# Train 1000 batches of 100 examples

for i in range(1000):
  batch = mnist.train.next_batch(100)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

In [16]:

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9174


That's is too low! Let's try a CNN
## CNN

In [17]:
# Weight initialization functions

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [18]:
# Convolution and pooling

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')